Fetching Webpages with the Requests Library To Demonstrate
===========================================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [23]:
from sqlalchemy import *
import mediacloud, datetime

In [34]:
mc = mediacloud.api.MediaCloud('b4aca10d383dab0dd274a84e7b29e678652f170a090ca4f412d3b98ec000db05')

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [42]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('sentencecount', Integer),
)
metadata.create_all(engine) # and create the tables in the database

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [43]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, sentencecount) VALUES (:id, :keywords, :timestamp, :sentencecount)'

In [44]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt)

'INSERT INTO queries (keywords, timestamp) VALUES (:keywords, :timestamp)'

In [45]:
query=('snakes')
res = mc.sentenceCount(query, solr_filter=[mc.publish_date_query( datetime.date( 2008, 1, 1), datetime.date( 2015, 12, 31) ), 'media_sets_id:1' ])
print res['count'] # prints the number of sentences found
count = res['count']
insert_stmt = queries.insert().values(keywords=terms, sentencecount=count)

23827


In [48]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[2]

In [46]:
query=('lizards')
res = mc.sentenceCount(query, solr_filter=[mc.publish_date_query( datetime.date( 2008, 1, 1), datetime.date( 2015, 12, 31) ), 'media_sets_id:1' ])
print res['count'] # prints the number of sentences found
count = res['count']
insert_stmt = queries.insert().values(keywords=terms, sentencecount=count)

6285


In [47]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[1]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [ ]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

In [49]:
select_stmt = select([queries]).where(queries.c.sentencecount)
sum = 0
for row in db_conn.execute(select_stmt):
    print row.sentencecount
    sum = sum + row.sentencecount
print "Number of sentences in US Mainstream Media that mention keywords: %i" %sum

6285
6285
Number of sentences in US Mainstream Media that mention keywords: 12570


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [ ]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [ ]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [ ]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [ ]:
query = Query(keywords="iguana")
query.keywords

In [ ]:
my_session.add(query)
my_session.commit()
query.id

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q